In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Load dataset
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "iammustafatz/diabetes-prediction-dataset",
    "diabetes_prediction_dataset.csv"
)

# Encode categorical variables
df = df.copy()
df['gender'] = df['gender'].astype('category').cat.codes
df['smoking_history'] = df['smoking_history'].astype('category').cat.codes

X = df.drop(columns=['diabetes'])
y = df['diabetes']

# Train (reference) vs deployment split
X_ref, X_deploy, y_ref, y_deploy = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# Scale using reference data ONLY
scaler = StandardScaler()
X_ref_scaled = scaler.fit_transform(X_ref)
X_deploy_scaled = scaler.transform(X_deploy)

# Simulate streaming batches
n_batches = 10
X_batches = np.array_split(X_deploy_scaled, n_batches)
y_batches = np.array_split(y_deploy.values, n_batches)

print("Reference samples:", X_ref_scaled.shape[0])
print("Deployment samples:", X_deploy_scaled.shape[0])
print("Batches:", len(X_batches))


/tmp/ipython-input-621625255.py:9: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 734k/734k [00:00<00:00, 1.12MB/s]

Extracting zip of diabetes_prediction_dataset.csv...


Reference samples: 60000
Deployment samples: 40000
Batches: 10


In [3]:
from scipy.stats import ks_2samp

# ---------- PSI FUNCTION ----------
def population_stability_index(ref, cur, bins=10):
    ref_hist, bin_edges = np.histogram(ref, bins=bins)
    cur_hist, _ = np.histogram(cur, bins=bin_edges)

    ref_perc = ref_hist / np.sum(ref_hist)
    cur_perc = cur_hist / np.sum(cur_hist)

    psi = np.sum(
        (ref_perc - cur_perc) * np.log((ref_perc + 1e-10) / (cur_perc + 1e-10))
    )
    return psi

# ---------- COMPUTE DRIFT PER BATCH ----------
psi_scores = []
ks_pvalues = []

for i, X_b in enumerate(X_batches):
    batch_psi = []
    batch_ks = []

    for f in range(X_ref_scaled.shape[1]):
        psi = population_stability_index(
            X_ref_scaled[:, f],
            X_b[:, f]
        )
        ks_stat, ks_p = ks_2samp(
            X_ref_scaled[:, f],
            X_b[:, f]
        )

        batch_psi.append(psi)
        batch_ks.append(ks_p)

    psi_scores.append(np.mean(batch_psi))
    ks_pvalues.append(np.mean(batch_ks))

    print(
        f"Batch {i+1}: "
        f"Mean PSI={np.mean(batch_psi):.4f}, "
        f"Mean KS p-value={np.mean(batch_ks):.4f}"
    )


Batch 1: Mean PSI=0.0022, Mean KS p-value=0.9541
Batch 2: Mean PSI=0.0020, Mean KS p-value=0.8612
Batch 3: Mean PSI=0.0022, Mean KS p-value=0.8079
Batch 4: Mean PSI=0.0023, Mean KS p-value=0.6788
Batch 5: Mean PSI=0.0025, Mean KS p-value=0.6828
Batch 6: Mean PSI=0.0020, Mean KS p-value=0.7713
Batch 7: Mean PSI=0.0016, Mean KS p-value=0.7846
Batch 8: Mean PSI=0.0016, Mean KS p-value=0.6895
Batch 9: Mean PSI=0.0008, Mean KS p-value=0.9189
Batch 10: Mean PSI=0.0013, Mean KS p-value=0.8228


In [4]:
# ---------- DRIFT INJECTION ----------
X_batches_drifted = []

for i, X_b in enumerate(X_batches):
    X_b_drifted = X_b.copy()

    # Inject drift after batch 5
    if i >= 5:
        # Shift important medical features
        X_b_drifted[:, 1] += 0.7   # glucose
        X_b_drifted[:, 5] += 0.4   # BMI
        X_b_drifted[:, 6] += 0.3   # HbA1c-like effect

        # Increase variance
        X_b_drifted += np.random.normal(0, 0.15, X_b_drifted.shape)

    X_batches_drifted.append(X_b_drifted)

print("Drift injected into batches 6–10")


Drift injected into batches 6–10


In [5]:
psi_scores_drifted = []
ks_pvalues_drifted = []

for i, X_b in enumerate(X_batches_drifted):
    batch_psi = []
    batch_ks = []

    for f in range(X_ref_scaled.shape[1]):
        psi = population_stability_index(
            X_ref_scaled[:, f],
            X_b[:, f]
        )
        ks_stat, ks_p = ks_2samp(
            X_ref_scaled[:, f],
            X_b[:, f]
        )

        batch_psi.append(psi)
        batch_ks.append(ks_p)

    psi_scores_drifted.append(np.mean(batch_psi))
    ks_pvalues_drifted.append(np.mean(batch_ks))

    print(
        f"Batch {i+1}: "
        f"Mean PSI={np.mean(batch_psi):.4f}, "
        f"Mean KS p-value={np.mean(batch_ks):.4f}"
    )


Batch 1: Mean PSI=0.0022, Mean KS p-value=0.9541
Batch 2: Mean PSI=0.0020, Mean KS p-value=0.8612
Batch 3: Mean PSI=0.0022, Mean KS p-value=0.8079
Batch 4: Mean PSI=0.0023, Mean KS p-value=0.6788
Batch 5: Mean PSI=0.0025, Mean KS p-value=0.6828
Batch 6: Mean PSI=2.5418, Mean KS p-value=0.0000
Batch 7: Mean PSI=2.4989, Mean KS p-value=0.0000
Batch 8: Mean PSI=2.6887, Mean KS p-value=0.0000
Batch 9: Mean PSI=2.6703, Mean KS p-value=0.0000
Batch 10: Mean PSI=2.6746, Mean KS p-value=0.0000


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score

# Train model on reference data
clf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced'
)

clf.fit(X_ref_scaled, y_ref)

batch_accuracy = []
batch_recall = []

for i, (X_b, y_b) in enumerate(zip(X_batches_drifted, y_batches)):
    y_pred = clf.predict(X_b)

    acc = accuracy_score(y_b, y_pred)
    rec = recall_score(y_b, y_pred)

    batch_accuracy.append(acc)
    batch_recall.append(rec)

    print(
        f"Batch {i+1}: "
        f"Accuracy={acc:.3f}, "
        f"Recall={rec:.3f}"
    )


Batch 1: Accuracy=0.973, Recall=0.721
Batch 2: Accuracy=0.975, Recall=0.720
Batch 3: Accuracy=0.971, Recall=0.704
Batch 4: Accuracy=0.970, Recall=0.696
Batch 5: Accuracy=0.969, Recall=0.673
Batch 6: Accuracy=0.834, Recall=0.786
Batch 7: Accuracy=0.844, Recall=0.766
Batch 8: Accuracy=0.842, Recall=0.757
Batch 9: Accuracy=0.846, Recall=0.801
Batch 10: Accuracy=0.841, Recall=0.805


In [7]:
# ---------- ALERTING LOGIC ----------
alerts = []

for i in range(n_batches):
    drift_alert = psi_scores_drifted[i] > 0.2
    performance_drop = batch_accuracy[i] < 0.90

    if drift_alert and not performance_drop:
        status = "⚠️ Silent Failure Risk"
    elif drift_alert and performance_drop:
        status = "🚨 Model Failure"
    else:
        status = "✅ Normal"

    alerts.append(status)

    print(
        f"Batch {i+1}: "
        f"PSI={psi_scores_drifted[i]:.2f}, "
        f"Accuracy={batch_accuracy[i]:.3f} → "
        f"{status}"
    )


Batch 1: PSI=0.00, Accuracy=0.973 → ✅ Normal
Batch 2: PSI=0.00, Accuracy=0.975 → ✅ Normal
Batch 3: PSI=0.00, Accuracy=0.971 → ✅ Normal
Batch 4: PSI=0.00, Accuracy=0.970 → ✅ Normal
Batch 5: PSI=0.00, Accuracy=0.969 → ✅ Normal
Batch 6: PSI=2.54, Accuracy=0.834 → 🚨 Model Failure
Batch 7: PSI=2.50, Accuracy=0.844 → 🚨 Model Failure
Batch 8: PSI=2.69, Accuracy=0.842 → 🚨 Model Failure
Batch 9: PSI=2.67, Accuracy=0.846 → 🚨 Model Failure
Batch 10: PSI=2.67, Accuracy=0.841 → 🚨 Model Failure
